In [1]:
import sys
import pandas as pd
import numpy as np
import nltk
from nltk.stem.porter import *
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import os
from sklearn.linear_model import LogisticRegression

In [2]:
path = "/Users/wonseok/Downloads/aclImdb/train/pos/"
temp = []
for file in os.listdir(path):
    with open(os.path.join(path + file), "r") as f:
        temp.append(f.read())
        
pos = pd.DataFrame({"reviews": temp, "labels": list(np.ones(len(temp), dtype=int))})

In [3]:
path = "/Users/wonseok/Downloads/aclImdb/train/neg/"
temp = []
for file in os.listdir(path):
    with open(os.path.join(path + file), "r") as f:
        temp.append(f.read())
        
neg = pd.DataFrame({"reviews": temp, "labels": list(np.zeros(len(temp), dtype=int))})

In [4]:
train_data = pd.concat([pos, neg], ignore_index=True)

In [5]:
path = "/Users/wonseok/Downloads/aclImdb/test/pos/"
temp = []
for file in os.listdir(path):
    with open(os.path.join(path + file), "r") as f:
        temp.append(f.read())
        
pos = pd.DataFrame({"reviews": temp, "labels": list(np.ones(len(temp), dtype=int))})

In [6]:
path = "/Users/wonseok/Downloads/aclImdb/test/neg/"
temp = []
for file in os.listdir(path):
    with open(os.path.join(path + file), "r") as f:
        temp.append(f.read())
        
neg = pd.DataFrame({"reviews": temp, "labels": list(np.zeros(len(temp), dtype=int))})

In [7]:
test_data = pd.concat([pos, neg], ignore_index=True)

In [8]:
def tokenize(text):
    """
    Tokenize text and return a non-unique list of tokenized words
    found in the text. Normalize to lowercase, strip punctuation,
    remove stop words, drop words of length < 3, strip digits.
    """
    text = text.lower()
    text = re.sub('[' + string.punctuation + '0-9\\r\\t\\n]', ' ', text)
    tokens = nltk.word_tokenize(text)
    tokens = [w for w in tokens if len(w) > 2]  # ignore a, an, to, at, be, ...
    tokens = [w for w in tokens if w not in ENGLISH_STOP_WORDS]
    return tokens

In [9]:
def stemwords(words):
    """
    Given a list of tokens/words, return a new list with each word
    stemmed using a PorterStemmer.
    """
    stemmer = PorterStemmer()
    stemmed = [stemmer.stem(w) for w in words]
    return stemmed

In [10]:
def tokenizer(text):
    return stemwords(tokenize(text))

In [11]:
def fit_corpus(train_data, test_data):
    corpus = pd.DataFrame({"reviews": train_data["reviews"]})
    corpus.reviews.append(test_data["reviews"], ignore_index=True)
    tfidf = TfidfVectorizer(input='content', analyzer='word', tokenizer=tokenizer, stop_words='english', decode_error = 'ignore')
    tfidf.fit(corpus["reviews"])
    return tfidf

In [12]:
def transform_data(tfidf, dataset):
    features = tfidf.transform(dataset["reviews"])
    return pd.DataFrame(features.todense(), columns = tfidf.get_feature_names())

In [13]:
tfidf = fit_corpus(train_data, test_data)

train_features = transform_data(tfidf, train_data) 
test_features = transform_data(tfidf, test_data)

train_labels = train_data["labels"]
test_labels = test_data["labels"]

/Users/wonseok/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [14]:
clf = LogisticRegression(random_state=0, solver='lbfgs')

In [15]:
clf.fit(train_features, train_labels)

LogisticRegression(random_state=0)

In [19]:
round(clf.score(test_features, test_labels), 2)

0.87